# Laboratorio 5 Parte 1

### Reducción de dimensión por selección de características

### Universidad de Antioquia

### Facultad de Ingeniería

### Ingeniería de Sistemas

### Ude@ - 2018-I

#### Profesor: Antonio Tamayo Herrera

## Estudiantes

Nombre: Rodrigo Antonio Porras Martinez

Céduda: 1039459720

Nombre: David Yepes Isaza

Cédula: 1020455542

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de selección de características y usar SVM para resolver un problema de clasificación multietiqueta o multiclase.

Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography

#### Abstract: 
The dataset consists of measurements of fetal heart rate (FHR) and uterine contraction (UC) features on cardiotocograms classified by expert obstetricians.
	

#### Data Set Information:

2126 fetal cardiotocograms (CTGs) were automatically processed and the respective diagnostic features measured. The CTGs were also classified by three expert obstetricians and a consensus classification label assigned to each of them. Classification was both with respect to a morphologic pattern (A, B, C. ...) and to a fetal state (N, S, P). Therefore the dataset can be used either for 10-class or 3-class experiments.

#### Attribute Information:

LB - FHR baseline (beats per minute)

AC - # of accelerations per second

FM - # of fetal movements per second

UC - # of uterine contractions per second

DL - # of light decelerations per second

DS - # of severe decelerations per second

DP - # of prolongued decelerations per second

ASTV - percentage of time with abnormal short term variability

MSTV - mean value of short term variability

ALTV - percentage of time with abnormal long term variability

MLTV - mean value of long term variability

Width - width of FHR histogram

Min - minimum of FHR histogram

Max - Maximum of FHR histogram

Nmax - # of histogram peaks

Nzeros - # of histogram zeros

Mode - histogram mode

Mean - histogram mean

Median - histogram median

Variance - histogram variance

Tendency - histogram tendency

CLASS - FHR pattern class code (1 to 10)

NSP - fetal state class code (N=normal (1); S=suspect (2); P=pathologic (3))


Analice la base de datos, sus características, su variable de salida y el contexto del problema.

Antes de iniciar a ejecutar las celdas, debe instalar la librería mlxtend que usaremos para los laboratorios de reducción de dimensión.
Para hacerlo solo tiene que usar el siguiente comando: sudo pip install mlxtend. También puede consultar la guía oficial de instalación
    de esta librería: https://rasbt.github.io/mlxtend/installation/

Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [4]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import time

#cargamos la bd de entrenamiento
db = np.loadtxt('DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

print ("Dimensiones de la base de datos de entrenamiento. dim de X: " + str(np.shape(X)) + "\tdim de Y: " + str(np.shape(Y)))


Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)


En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [5]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

#Para calcular el costo computacional
tiempo_i = time.time()

#Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

#Implemetamos la metodología de validación

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    #Aquí se entran y se valida el modelo sin hacer selección de características
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1

print("\nError de validación sin aplicar SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

#print str(ypred)
#print str(y_test) 


Error de validación sin aplicar SFS: 0.07712817787226504 +/- 0.05442325724156325


Tiempo total de ejecución: 0.4618513584136963 segundos.


## Ejercicio 1

1.1 Describa la metodología de validación que se está aplicando.

R/: La metodologia de validacion que se esta usando es CrossValidation con 10 Folds.
    
1.2 Con qué modelo se está resolviendo el problema planteado? Cuáles son los parámetros establecidos para el modelo?

R/:El modelo a usar es SVM (Maquinas de vectores de Soporte), para este caso como es clasificacion un SVC que es un clasificador multiclase. El parametro decision_function_shape='ovr' es la estrategia para clasificacion multiclase "one vs rest" o "one vs all". El kernel='rbf' (Radial basis function) sirve para llevar el problema a un espacio de mayor dimensión donde si se puedan separar "linealmente" las muestras. El parametro "C" es el parametro de penalizacion del error. Gamma es el coeficiente del kernel RBF.

## Ejercicio 2

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de selección de características. https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#sequential-feature-selector

In [6]:
#Feature Selection Function
#Recibe 4 parámetros: 1. el modelo (clf para nuestro caso), 2. el número de características final que se quiere alcanzar
#3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False
def select_features(modelo, n_features, fwd, fltg):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0)
    
    return sfs


#Para calcular el costo computacional
tiempo_i = time.time()

#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
   
    #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias
    
    #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
    #Ejercicio 3.1
    sf = select_features(clf,10,False,False)

    #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
    #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
    sf = sf.fit(X_train,y_train)
    
    Errores[j] = 1-sf.k_score_
    j+=1

print("\nError de validación aplicando SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print("\nEficiencia en validación aplicando SFS: " + str(sf.k_score_*100) + "%" )

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

print("Las caracteristicas seleccionadas son: ")

dictChar = {0 : "LB - FHR baseline (beats per minute)", 
            1 : 'AC - # of accelerations per second',
            2 : 'FM - # of fetal movements per second',
            3 : 'UC - # of uterine contractions per second',
            4 : 'DL - # of light decelerations per second',
            5 : 'DS - # of severe decelerations per second',
            6 : 'DP - # of prolongued decelerations per second',
            7 : 'ASTV - percentage of time with abnormal short term variability',
            8 : 'MSTV - mean value of short term variability',
            9 : 'ALTV - percentage of time with abnormal long term variability',
            10 :'MLTV - mean value of long term variability',
            11 :'Width - width of FHR histogram',
            12 :'Min - minimum of FHR histogram',
            13 :'Max - Maximum of FHR histogram',
            14 :'Nmax - # of histogram peaks',
            15 :'Nzeros - # of histogram zeros',
            16 :'Mode - histogram mode',
            17 :'Mean - histogram mean',
            18 :'Median - histogram median',
            19 :'Variance - histogram variance',
            20 :'Tendency - histogram tendency',
            21 :'CLASS - FHR pattern class code (1 to 10)'}

for x in range(0,len(sf.k_feature_idx_)):
    print (dictChar[sf.k_feature_idx_[x]])

[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    1.2s finished
Features: 21/10[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.1s finished
Features: 20/10[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.0s finished
Features: 19/10[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.9s finished
Features: 18/10[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.9s finished
Features: 17/10[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.8s finished
Features: 16/10[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.7s finished
Features: 15/10[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.7s finished
Features: 14/10[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    0.6s finished
Features: 13/10[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.5s finished
Features: 12/10[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.5s finished
Features: 11/10[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.4s fini

Features: 21/10[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.2s finished
Features: 20/10[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.1s finished
Features: 19/10[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.1s finished
Features: 18/10[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.0s finished
Features: 17/10[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.9s finished
Features: 16/10[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.8s finished
Features: 15/10[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    0.8s finished
Features: 14/10[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    0.7s finished
Features: 13/10[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.6s finished
Features: 12/10[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.5s finished
Features: 11/10[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.5s finished
Features: 10/10[Parallel(n_jobs=1)]: Done  22 out of  22 | elapse


Error de validación aplicando SFS: 0.011288625753178627 +/- 0.001282860778208819

Eficiencia en validación aplicando SFS: 98.90282131661442%


Tiempo total de ejecución: 115.82241439819336 segundos.
Las caracteristicas seleccionadas son: 
LB - FHR baseline (beats per minute)
AC - # of accelerations per second
FM - # of fetal movements per second
DL - # of light decelerations per second
ASTV - percentage of time with abnormal short term variability
ALTV - percentage of time with abnormal long term variability
Min - minimum of FHR histogram
Median - histogram median
Variance - histogram variance
CLASS - FHR pattern class code (1 to 10)


## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de características a seleccionar (use 3, 7 y 10) y la estrategia a implementar (SFS, SBS, SFFS, SBFS), para que complete la siguiente tabla de resultados:

<table>
  <tr>
    <th>Técnica</th>
    <th># de características seleccionadas</th>
    <th>Error de validación</th>
    <th>IC (std)</th>
    <th>Tiempo de ejecución</th>
  </tr>
  <tr>
    <td>SVM sin selección</td>
    <td>22</td>
    <td>0.07712817787226504 +/- 0.05442325724156325</td>
    <td></td>
    <td>0.43585944175720215</td>
  </tr>
  <tr>
    <td>SVM + SFS</td>
    <td>3</td>
    <td>0.04311674890112802 +/- 0.005501557850665672</td>
    <td>95.61128526645768%</td>
    <td>51.4155158996582</td>
  </tr>
  <tr>
    <td>SVM + SFS</td>
    <td>7</td>
    <td>0.019911828052138435 +/- 0.002727682121130486</td>
    <td>97.91013584117032%</td>
    <td>87.94931316375732</td>
  </tr>
  <tr>
    <td>SVM + SFS</td>
    <td>10</td>
    <td>0.01306550189240313 +/- 0.002347757559935403</td>
    <td>98.58934169278997%</td>
    <td>107.52703142166138</td>
  </tr>
  <tr>
    <td>SVM + SBS</td>
    <td>3</td>
    <td>0.044214200697968754 +/- 0.004748624592322402</td>
    <td>95.141065830721%</td>
    <td>137.60688376426697</td>
  </tr>
  <tr>
    <td>SVM + SBS</td>
    <td>7</td>
    <td>0.01991207385424808 +/- 0.0026168019256843054</td>
    <td>98.27586206896551%</td>
    <td>123.75583028793335</td>
  </tr>
  <tr bgcolor="#40FF00">
    <td>SVM + SBS</td>
    <td>10</td>
    <td>0.011288625753178627 +/- 0.001282860778208819</td>
    <td>98.90282131661442%</td>
    <td>110.2136766910553</td>
  </tr>
  <tr>
    <td>SVM + SFFS</td>
    <td>3</td>
    <td>0.04311674890112802 +/- 0.005501557850665672</td>
    <td>95.61128526645768%</td>
    <td>54.392578125</td>
  </tr>
  <tr >
    <td>SVM + SFFS</td>
    <td>7</td>
    <td>0.019911828052138435 +/- 0.002727682121130486</td>
    <td>97.91013584117032%</td>
    <td>102.69758009910583</td>
  </tr>
  <tr>
    <td>SVM + SFFS</td>
    <td>10</td>
    <td>0.012804132315821847 +/- 0.0022432590153498955</td>
    <td>98.58934169278997%</td>
    <td>137.75034403800964</td>
  </tr>
  <tr>
    <td>SVM + SBFS</td>
    <td>3</td>
    <td>0.044475734142623125 +/- 0.004775170586947885</td>
    <td>95.61128526645768%</td>
    <td>328.1763141155243</td>
  </tr>
  <tr>
    <td>SVM + SBFS</td>
    <td>7</td>
    <td>0.018866923284069137 +/- 0.002543983180019345</td>
    <td>98.27586206896551%</td>
    <td>230.11173295974731</td>
  </tr>
  <tr >
    <td>SVM + SBFS</td>
    <td>10</td>
    <td>0.01102731079928836 +/- 0.0015918971954240022</td>
    <td>98.90282131661442%</td>
    <td>180.62711119651794</td>
  </tr>    
</table>


3.2 Según la teoría vista en el curso, se está usando una función tipo filtro o tipo wrapper y cuál es?

R/: Se esta usando una funcion tipo wrapper, la funcion objetivo es SVC con una metodologia de validacion CROSS-VALIDATION.

3.3 Con los resultados de la tabla anterior haga un análisis juicioso de cuál es el mejor resultado teniendo en cuenta tanto la eficiencia en la clasificación como el costo computacional del modelo y la estrategia implementada.

R/: Para nuestro caso el mejor resultado se con SVM + SBS (Sequential Backward Selection) selecionando 10 caracteristicas tiene la mejor eficiencia con un 98.9% y con respecto al costo computacional tiene un tiempo de ejecuccion de 110.2 s. Lo que hace que su costo  computacional sea por debajo de promedio de los resultados. En conclusión, obtenemos la mayor eficiencia con menos tiempo.

3.4 Haga uso del atributo sf.k_feature\_idx\_ (deje evidencia del código usado para esto) para identificar cuáles fueron las características seleccionadas en el mejor de los resultados encontrados. No presente los indices de las características sino sus nombres y descripción.

R/: Las caracteristicas seleccionadas son: 
LB - FHR baseline (beats per minute)
AC - # of accelerations per second
FM - # of fetal movements per second
DL - # of light decelerations per second
ASTV - percentage of time with abnormal short term variability
ALTV - percentage of time with abnormal long term variability
Min - minimum of FHR histogram
Median - histogram median
Variance - histogram variance
CLASS - FHR pattern class code (1 to 10)

3.5 De acuerdo a los resultados encontrados y la respuesta anterior, usted como ingeniero de datos que le puede sugerir a un médico que esté trabajando en un caso enmarcado dentro del contexto de la base de datos trabajada, para que apoye su diagnóstico?

R/: Como ingenieros de datos le mostramos la validacion que se le realiza al modelo, se recomienda que los resultados son un apoyo a su Diagnostico permitiendole ayudar de manera temprana a sus pacientes en gestación. Con las anteriores caracteristicas  puede usted reducir el tamaño de su Base de datos y aun asi obtener buenos resultados. Ademas, solo tendria que tomar informacion de 10 caracteristicas y no de 23, reduciendo las tiempo de toma de datos y dedicarlo al analisis de resultados. Para el paciente significa menos tiempo en medico, consultas mas eficientes.  